In [1]:
# Celda 1: Importaciones Necesarias
import requests
import json
import os

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [3]:
# Celda 2: Configuración Global
BASE_URL = "http://localhost:8000" # Asegúrate de que el puerto coincida

# Rutas de los Endpoints
OCR_ENDPOINT_URL = f"{BASE_URL}/convert-pdf"
CLASSIFY_GENERAL_URL = f"{BASE_URL}/clasificar-texto"
CLASSIFY_FACTURE_URL = f"{BASE_URL}/clasificar-factura-examenes"

# Nombre del archivo PDF de prueba (colócalo en el mismo directorio que este notebook)
PDF_FILENAME_FOR_OCR = "NAN_FEHP701932.pdf"
pdf_file_path_for_ocr = os.path.join(os.getcwd(), PDF_FILENAME_FOR_OCR)

# Variable global para almacenar el texto extraído por OCR para usarlo en otras pruebas
extracted_ocr_text_global = None

# Verificar si el archivo PDF de prueba existe
pdf_existe_para_ocr = os.path.exists(pdf_file_path_for_ocr)
if not pdf_existe_para_ocr:
    print(f"¡ADVERTENCIA! El archivo PDF de prueba '{pdf_file_path_for_ocr}' no fue encontrado.")
    print("La prueba del endpoint /convert-pdf y las pruebas dependientes del OCR se omitirán o fallarán.")
else:
    print(f"Archivo PDF de prueba para OCR encontrado en: {pdf_file_path_for_ocr}")

print(f"\nConfiguración de API:")
print(f"  OCR Endpoint: {OCR_ENDPOINT_URL}")
print(f"  Clasificación General Endpoint: {CLASSIFY_GENERAL_URL}")
print(f"  Clasificación Facturas Endpoint: {CLASSIFY_FACTURE_URL}")

Archivo PDF de prueba para OCR encontrado en: c:\repositorios\pdf2Image\NAN_FEHP701932.pdf

Configuración de API:
  OCR Endpoint: http://localhost:8000/convert-pdf
  Clasificación General Endpoint: http://localhost:8000/clasificar-texto
  Clasificación Facturas Endpoint: http://localhost:8000/clasificar-factura-examenes


In [4]:
# Celda 3: Función Auxiliar para Realizar Solicitudes POST JSON
def post_json_request(endpoint_url: str, payload: dict, test_name: str):
    """Realiza una solicitud POST con cuerpo JSON y maneja la respuesta."""
    print(f"\n--- Probando: {test_name} ---")
    print(f"URL: {endpoint_url}")
    print(f"Payload (primeros 100 chars del texto si existe):")
    if "texto_documento" in payload:
        print(f"  {{'texto_documento': '{str(payload['texto_documento'])[:100].replace('\n',' ')}...'}}")
    else:
        print(f"  {str(payload)[:200]}")

    try:
        response = requests.post(endpoint_url, json=payload)
        response.raise_for_status()  # Lanza excepción para errores HTTP 4xx/5xx
        
        response_data = response.json()
        print(f"Respuesta API (Estado {response.status_code}):")
        print(json.dumps(response_data, indent=2, ensure_ascii=False))
        return response_data
        
    except requests.exceptions.ConnectionError:
        print(f"[ERROR DE CONEXIÓN] No se pudo conectar a la API en {BASE_URL}.")
        print("Asegúrate de que el servidor FastAPI (uvicorn) esté ejecutándose.")
    except requests.exceptions.HTTPError as http_err:
        print(f"[ERROR HTTP: {http_err.response.status_code}] Respuesta del servidor: {http_err.response.text}")
    except Exception as e:
        print(f"[ERROR INESPERADO] Ocurrió un error: {e}")
        import traceback
        traceback.print_exc()
    return None # Retorna None si hay error

In [5]:
# Celda 4: Prueba del Endpoint /convert-pdf (Extracción OCR)

print("\n" + "="*10 + " PRUEBA ENDPOINT: /convert-pdf " + "="*10)

if pdf_existe_para_ocr:
    test_name_ocr = f"Subir PDF '{PDF_FILENAME_FOR_OCR}' para OCR"
    print(f"\n--- Probando: {test_name_ocr} ---")
    print(f"URL: {OCR_ENDPOINT_URL}")
    
    try:
        with open(pdf_file_path_for_ocr, 'rb') as f:
            files_payload = {'file': (PDF_FILENAME_FOR_OCR, f, 'application/pdf')}
            response_ocr = requests.post(OCR_ENDPOINT_URL, files=files_payload)
            response_ocr.raise_for_status()
            
            ocr_data = response_ocr.json()
            print(f"Respuesta API (Estado {response_ocr.status_code}):")
            # print(json.dumps(ocr_data, indent=2, ensure_ascii=False)) # Puede ser muy largo
            
            extracted_ocr_text_global = ocr_data.get('texto_completo_ocr', '')
            print(f"Texto Completo OCR (primeros 200 caracteres):\n'{extracted_ocr_text_global[:200]}...'")
            
            if extracted_ocr_text_global:
                 print("✅ OCR Exitoso y texto extraído.")
            else:
                 print("⚠️ OCR Posiblemente exitoso, pero no se extrajo texto.")

    except requests.exceptions.ConnectionError:
        print(f"[ERROR DE CONEXIÓN] No se pudo conectar a la API en {BASE_URL}.")
    except requests.exceptions.HTTPError as http_err:
        print(f"[ERROR HTTP: {http_err.response.status_code}] Respuesta del servidor: {http_err.response.text}")
    except Exception as e:
        print(f"[ERROR INESPERADO] Ocurrió un error: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"OMITIDO: Prueba del endpoint /convert-pdf porque el archivo '{PDF_FILENAME_FOR_OCR}' no fue encontrado.")


========== PRUEBA ENDPOINT: /convert-pdf ==========

--- Probando: Subir PDF 'NAN_FEHP701932.pdf' para OCR ---
URL: http://localhost:8000/convert-pdf
[ERROR HTTP: 422] Respuesta del servidor: {"detail":[{"type":"missing","loc":["query","texto_documento"],"msg":"Field required","input":null}]}


In [6]:
# Celda 5: Pruebas del Endpoint /clasificar-texto (Clasificación General)

print("\n" + "="*10 + " PRUEBA ENDPOINT: /clasificar-texto " + "="*10)

# Casos de prueba para clasificación general
casos_general = [
    {"nombre": "Autorización Típica", "texto": "AUTORIZACION DE SERVICIOS Pagina 1 de 1."},
    {"nombre": "Orden Médica con Diagnóstico", "texto": "PLAN DE MANEJO: Paciente requiere consulta especializada. Diagnóstico: N40X."},
    {"nombre": "Soporte de Recibido", "texto": "FORMATO DE RECIBIDO USUARIO. Certifico que recibí a satisfacción."},
    {"nombre": "Texto sin Clasificación Clara", "texto": "Hoja de evolución del paciente. Signos vitales estables."},
    {"nombre": "Texto Vacío", "texto": ""}
]

# Añadir el texto del OCR si se extrajo
if extracted_ocr_text_global:
    casos_general.append({"nombre": f"Texto del OCR de '{PDF_FILENAME_FOR_OCR}'", "texto": extracted_ocr_text_global})

for caso in casos_general:
    payload = {"texto_documento": caso["texto"]}
    resultado_general = post_json_request(CLASSIFY_GENERAL_URL, payload, f"Clasificación General - {caso['nombre']}")
    # Aquí podrías añadir aserciones si conoces el resultado esperado
    # Ejemplo: assert resultado_general.get("clasificacion_reglas") == "autorizacion", f"Falló {caso['nombre']}"


========== PRUEBA ENDPOINT: /clasificar-texto ==========

--- Probando: Clasificación General - Autorización Típica ---
URL: http://localhost:8000/clasificar-texto
Payload (primeros 100 chars del texto si existe):
  {'texto_documento': 'AUTORIZACION DE SERVICIOS Pagina 1 de 1....'}
[ERROR HTTP: 404] Respuesta del servidor: {"detail":"Not Found"}

--- Probando: Clasificación General - Orden Médica con Diagnóstico ---
URL: http://localhost:8000/clasificar-texto
Payload (primeros 100 chars del texto si existe):
  {'texto_documento': 'PLAN DE MANEJO: Paciente requiere consulta especializada. Diagnóstico: N40X....'}
[ERROR HTTP: 404] Respuesta del servidor: {"detail":"Not Found"}

--- Probando: Clasificación General - Soporte de Recibido ---
URL: http://localhost:8000/clasificar-texto
Payload (primeros 100 chars del texto si existe):
  {'texto_documento': 'FORMATO DE RECIBIDO USUARIO. Certifico que recibí a satisfacción....'}
[ERROR HTTP: 404] Respuesta del servidor: {"detail":"Not Found"}

In [7]:
# Celda 6: Pruebas del Endpoint /clasificar-factura-examenes

print("\n" + "="*10 + " PRUEBA ENDPOINT: /clasificar-factura-examenes " + "="*10)

# Casos de prueba para clasificación de facturas
casos_facturas = [
    {"nombre": "Factura con Examen (Radiografía)", "texto": "CUPS NOMBRE CANT VR UNIT\nPROCEDIMIENTOS DIAGNOSTICO\n873412 RADIOGRAFIA DE CADERA"},
    {"nombre": "Factura Solo Consulta", "texto": "CUPS NOMBRE CANT VR UNIT\nCONSULTAS\n39143OT CONSULTA DE PRIMERA VEZ OTORRINO"},
    {"nombre": "Factura Texto No Médico", "texto": "Supermercado El Ahorro\nManzanas 5 5.00"},
    {"nombre": "Factura Texto Vacío", "texto": ""}
]

# Añadir el texto del OCR si se extrajo y se cree que podría ser una factura
if extracted_ocr_text_global:
    # Puedes decidir si quieres probar el texto del OCR también con este clasificador
    # Dependerá del contenido de tu 'ejemplo_documento.pdf'
    casos_facturas.append({"nombre": f"Texto del OCR de '{PDF_FILENAME_FOR_OCR}' (como factura)", "texto": extracted_ocr_text_global})


for caso in casos_facturas:
    payload = {"texto_documento": caso["texto"]}
    resultado_factura = post_json_request(CLASSIFY_FACTURE_URL, payload, f"Clasificación Factura Exámenes - {caso['nombre']}")
    # Ejemplo de aserción:
    # if "Examen" in caso['nombre']:
    #     assert resultado_factura.get("examenesFacturados") == 1, f"Falló {caso['nombre']}"
    # elif "Consulta" in caso['nombre'] or "No Médico" in caso['nombre'] or "Vacío" in caso['nombre']:
    #     assert resultado_factura.get("examenesFacturados") == 0, f"Falló {caso['nombre']}"

print("\n" + "="*10 + " FIN DE TODAS LAS PRUEBAS " + "="*10)


========== PRUEBA ENDPOINT: /clasificar-factura-examenes ==========

--- Probando: Clasificación Factura Exámenes - Factura con Examen (Radiografía) ---
URL: http://localhost:8000/clasificar-factura-examenes
Payload (primeros 100 chars del texto si existe):
  {'texto_documento': 'CUPS NOMBRE CANT VR UNIT PROCEDIMIENTOS DIAGNOSTICO 873412 RADIOGRAFIA DE CADERA...'}
Respuesta API (Estado 200):
{
  "examenesFacturados": 1,
  "decision_source": "rules",
  "texto_limpio": "cups nombre cant vr unit procedimientos diagnostico 873412 radiografia de cadera"
}

--- Probando: Clasificación Factura Exámenes - Factura Solo Consulta ---
URL: http://localhost:8000/clasificar-factura-examenes
Payload (primeros 100 chars del texto si existe):
  {'texto_documento': 'CUPS NOMBRE CANT VR UNIT CONSULTAS 39143OT CONSULTA DE PRIMERA VEZ OTORRINO...'}
Respuesta API (Estado 200):
{
  "examenesFacturados": 0,
  "decision_source": "rules_consultas",
  "texto_limpio": "cups nombre cant vr unit consultas 39143ot 